# Debugging DQN_9

DQN_9 was the best performing model (on the F1-score) saved by Optuna optimisation. However during validation, the results are poor. This notebook troubleshoots that by re-training the model again with the best parameters. Turns out the problem is the best performing model wasn't saved correctly.

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from os.path import exists

sys.path.append('..')
sys.path.append('../..')

In [3]:
import copy

from experiments import preset_qcb_small
from debugging import debug_run, eval_res_to_df


## Generate environment preset

In [4]:
alpha = 0.191500954
beta = 0.030798858
extract = False
params, max_peaks = preset_qcb_small(None, alpha=alpha, beta=beta, extract_chromatograms=extract)
params, max_peaks

2022-08-23 13:53:15.367 | INFO     | experiments:get_samplers:283 - Loaded /Users/joewandy/Work/git/vimms-gym/pickles/samplers_QCB_small_gaussian.p


({'chemical_creator': {'mz_range': (100, 110),
   'rt_range': (400, 500),
   'intensity_range': (10000.0, 1e+20),
   'n_chemicals': (20, 50),
   'mz_sampler': <vimms.ChemicalSamplers.MZMLFormulaSampler at 0x7fda863cb100>,
   'ri_sampler': <vimms.ChemicalSamplers.MZMLRTandIntensitySampler at 0x7fda863cb2e0>,
   'cr_sampler': <vimms.ChemicalSamplers.GaussianChromatogramSampler at 0x7fda863d2c40>},
  'noise': {'enable_spike_noise': True,
   'noise_density': 0.1,
   'noise_max_val': 1000.0,
   'mz_range': (100, 110)},
  'env': {'ionisation_mode': 'Positive',
   'rt_range': (400, 500),
   'isolation_window': 0.7,
   'mz_tol': 10,
   'rt_tol': 120,
   'alpha': 0.191500954,
   'beta': 0.030798858}},
 100)

In [5]:
n_eval_episodes = 1

## Load best saved model from Optuna

In [6]:
fname = os.path.join('..', 'DQN', 'DQN_9.zip') 

### Deterministic = True

In [7]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=True)
df = eval_res_to_df(rewards, eval_res)
df

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.32GB

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/dqn/dqn.py:152: UserWarning:

The number of environments used is greater than the target network update interval (20 > 1), therefore the target network will be updated after each call to env.step() which corresponds to 20 steps.



{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}
Evaluation episode 0 finished: metric -499.000000, timedelta=0:00:00.371340


,metric,mean,std
0,reward,-499.0,0.0
1,coverage_prop,0.0,0.0
2,intensity_prop,0.0,0.0
3,f1,0.0,0.0


### Deterministic = False

In [8]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=False)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.32GB

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/dqn/dqn.py:152: UserWarning:

The number of environments used is greater than the target network update interval (20 > 1), therefore the target network will be updated after each call to env.step() which corresponds to 20 steps.



Evaluation episode 0 finished: metric -458.906496, timedelta=0:00:00.470321


,metric,mean,std
0,reward,-458.906496,0.0
1,coverage_prop,0.734694,0.0
2,intensity_prop,0.443456,0.0
3,f1,0.185185,0.0


## Load first re-trained model

In [9]:
fname = os.path.join('..', 'DQN', 'DQN_9_rerun_1.zip') 

### Deterministic = True

In [10]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=True)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.32GB



Evaluation episode 0 finished: metric 140.893994, timedelta=0:00:02.309373


,metric,mean,std
0,reward,140.893994,0.0
1,coverage_prop,0.923077,0.0
2,intensity_prop,0.837959,0.0
3,f1,0.555556,0.0


### Deterministic = False

In [11]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=False)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.33GB



Evaluation episode 0 finished: metric 54.626509, timedelta=0:00:01.556748


,metric,mean,std
0,reward,54.626509,0.0
1,coverage_prop,0.916667,0.0
2,intensity_prop,0.883556,0.0
3,f1,0.702703,0.0


## Load second re-trained model

In [12]:
fname = os.path.join('..', 'DQN', 'DQN_9_rerun_2.zip') 

### Deterministic = True

In [13]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=True)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.34GB



Evaluation episode 0 finished: metric 117.478518, timedelta=0:00:01.929983


,metric,mean,std
0,reward,117.478518,0.0
1,coverage_prop,0.964286,0.0
2,intensity_prop,0.750224,0.0
3,f1,0.526316,0.0


### Deterministic = False

In [14]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=False)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.31GB



Evaluation episode 0 finished: metric 56.616965, timedelta=0:00:02.805987


,metric,mean,std
0,reward,56.616965,0.0
1,coverage_prop,1.000000,0.0
2,intensity_prop,0.858244,0.0
3,f1,0.461538,0.0


## Load third re-trained model

In [15]:
fname = os.path.join('..', 'DQN', 'DQN_9_rerun_3.zip') 

### Deterministic = True

In [16]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=True)
df = eval_res_to_df(rewards, eval_res)
df

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.32GB



{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}
Evaluation episode 0 finished: metric 133.707996, timedelta=0:00:02.916000


,metric,mean,std
0,reward,133.707996,0.0
1,coverage_prop,1.000000,0.0
2,intensity_prop,0.874501,0.0
3,f1,0.452830,0.0


### Deterministic = False

In [17]:
rewards, eval_res = debug_run(fname, max_peaks, params, n_eval_episodes=n_eval_episodes, deterministic=False)
df = eval_res_to_df(rewards, eval_res)
df

{'ionisation_mode': 'Positive', 'rt_range': (400, 500), 'isolation_window': 0.7, 'mz_tol': 10, 'rt_tol': 120, 'alpha': 0.191500954, 'beta': 0.030798858}


/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 9.87GB > 0.31GB



Evaluation episode 0 finished: metric 24.578408, timedelta=0:00:02.234654


,metric,mean,std
0,reward,24.578408,0.0
1,coverage_prop,0.952381,0.0
2,intensity_prop,0.893814,0.0
3,f1,0.551724,0.0
